# Data engineering

In [1]:
from __future__ import print_function
from pandas_profiling import ProfileReport
from dateutil.parser import parse
from ipywidgets import interact

import matplotlib.pyplot as plt
import multiprocessing as mp
import ipywidgets as widgets
import plotly.express as px
import missingno as msno
import pandas as pd
import unicodedata
import numpy as np
import hashlib
import glob
import re

## Variables

In [2]:
RAW_DATA = './downloads/raw'
PPD_DATA = './downloads/pre-processed'

## Functions

Replace function

- df: input dataframe
- rules: substitution rules `{index: {'cols': columns, 'replaces': {value: new_value}, 'regex': bool}}` eg:
    - `{0: {'cols': 'Gender', 'replaces': {None:0.0}, 'regex': True}}`

In [3]:
def replace(df: pd.DataFrame, rules: {}) -> pd.DataFrame:    
    for rule in rules.values():
        cols = rule['cols']
        df[cols] = df[cols].replace(rule['replaces'], regex=rule['regex'])
    return df

## Visualization

In [4]:
df = pd.read_csv(f'{PPD_DATA}/all_wcoords.csv', low_memory=False)
df

,Adinamia (fraqueza),Alteração de ausculta pulmonar,Alteração na radiologia de tórax,Apresenta sintomas?,Bairro,CEP,Cefaleia (dor de cabeça),"Complemento ( Condomínio, edifício, apto)",Conjuntivite,Coriza,...,"Se teve contato com outro caso, favor especificar o local:",Sem morbidades,Sexo,Situação de saúde do paciente no momento da notificação:*,Tosse,[outros] Condições:Selecionar todas morbidades pertinentes.,[outros] Selecione os sinais clínicos observados:,geocode,lat,lon
0,NaN,NaN,NaN,1.0,ONDINA,40170040.0,NaN,"CONDOMÍNIO ONDINA TROPICAL, APTO 108",NaN,NaN,...,Local de trabalho,1.0,1.0,2.0,NaN,NaN,1.0,"[{'address_components': [{'long_name': '136', ...",-13.007811,-38.515199
1,NaN,NaN,NaN,2.0,pituacu,41741578.0,NaN,casa 10,NaN,NaN,...,NaN,1.0,1.0,NaN,NaN,NaN,NaN,[{'address_components': [{'long_name': '3A Tra...,-12.962164,-38.407345
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,2.0,Patamares,41680066.0,NaN,"Cond. Salvador Ville, Apt 106, Torre 2",NaN,NaN,...,NaN,NaN,1.0,2.0,NaN,NaN,NaN,[{'address_components': [{'long_name': 'Rua Ib...,-12.940893,-38.411320
4,NaN,NaN,NaN,2.0,Patamares,41680066.0,NaN,"Cond. Salvador Ville, Apt 106, Torre 2",NaN,NaN,...,NaN,NaN,1.0,2.0,NaN,NaN,NaN,[{'address_components': [{'long_name': 'Rua Ib...,-12.940893,-38.411320
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8631,NaN,NaN,NaN,2.0,VILAS ABRANTES,42800000.0,NaN,"COND ILHA DE BORACAY, CASA 10",NaN,NaN,...,Desconhecido,1.0,1.0,NaN,NaN,NaN,NaN,[{'address_components': [{'long_name': 'Rua Lu...,-12.727920,-38.329150
8632,NaN,NaN,NaN,2.0,LAURO DE FREITAS,42700000.0,NaN,CASA,NaN,NaN,...,NaN,1.0,1.0,NaN,NaN,NaN,NaN,[{'address_components': [{'long_name': 'Caminh...,-12.890887,-38.351356
8633,1.0,NaN,NaN,1.0,BARRA DE JACUIPE,42833000.0,1.0,COND. CAMINHO DO RIO,NaN,1.0,...,NaN,1.0,1.0,NaN,NaN,NaN,NaN,[{'address_components': [{'long_name': '42833-...,-12.699402,-38.131821
8634,NaN,NaN,NaN,2.0,Catu de Abrantes,4284000.0,NaN,Condominio Vivendas do Jones,NaN,NaN,...,NaN,1.0,1.0,NaN,NaN,NaN,NaN,[{'address_components': [{'long_name': 'Avenid...,-12.837399,-38.263647


In [5]:
rules = {
    0: {
        'cols': 'Sexo',
        'replaces': {1: 'Masculino', 2: 'Feminino'},
        'regex': False
    },
    1: {
        'cols': 'Raça',
        'replaces': {
            1: 'amarela',
            2: 'branca',
            3: 'indigena',
            4: 'parda',
            5: 'preta'
        },
        'regex': False
    },
    2: {
        'cols': [
            'Apresenta sintomas?',
            'Paciente foi hospitalizado?',
            'Paciente foi submetido a ventilação mecânica?',
        ],
        'replaces': {1: 'sim', 2: 'não', np.nan: 'nan'},
        'regex': False
    },
    3: {
        'cols': 'RESULTADO',
        'replaces': {1: 'Positivo', 2: 'Negativo', 9: 'Inconclusivo'},
        'regex': False
    }
}

df = replace(df, rules)

In [72]:
[c for c in df.columns]

['Adinamia (fraqueza)',
 'Alteração de ausculta pulmonar',
 'Alteração na radiologia de tórax',
 'Apresenta sintomas?',
 'Bairro',
 'CEP',
 'Cefaleia (dor de cabeça)',
 'Complemento ( Condomínio, edifício, apto)',
 'Conjuntivite',
 'Coriza',
 'DATA DA COLETA',
 'Data da alta hospitalar:',
 'Data da internação hospitalar:',
 'Data da viagem de ida para outro local transmissão:',
 'Data da viagem de volta do outro local transmissão:',
 'Data de nascimento',
 'Data do isolamento:',
 'Data dos primeiros sintomas:',
 'Descritivo do histórico de deslocamento nos 14 dias antes do início dos sintomas:',
 'Diabetes',
 'Diarreia',
 'Dificuldade de respirar',
 'Dispneia/Taquipneia',
 'Doença Respiratória crônica',
 'Doença cardiovascular, incluindo hipertensão',
 'Doença hepática',
 'Doença neurológica crônica ou neuromuscular',
 'Doença renal',
 'Doença respiratória crônica',
 'Dor de garganta',
 'Estado',
 'Esteve em alguma unidade de saúde nos 14 dias antes do início dos sintomas?*',
 'Exsudat

In [116]:
symptoms = [
    'Adinamia (fraqueza)',
    'Cefaleia (dor de cabeça)',
    'Coriza',
    'Diarreia',
    'Dificuldade de respirar',
    'Dispneia/Taquipneia',
    'Dor de garganta',
    'Exsudato faríngeo',
    'Febre',
    'Irritabilidade/confusão',
    'Neoplasia (tumor sólido ou hematológico)',
    'Náusea/vômitos',
    'Tosse',
]
comorbidities = [
    'Alteração de ausculta pulmonar',
    'Alteração na radiologia de tórax',
    'Conjuntivite',
    'Diabetes',
    'Doença Respiratória crônica',
    'Doença cardiovascular, incluindo hipertensão',
    'Doença hepática',
    'Doença neurológica crônica ou neuromuscular',
    'Doença renal',
    'Doença respiratória crônica',
    'Gestante',
    'Gestante de alto risco',
    'Imunodeficiência',
    'Imunossupressão',
    'Infecção pelo HIV',
    'Portador de alterações cromossômicas',
    'Portador de doenças cromossômicas',
    'Sem morbidades',
]

_vall = [*symptoms, *comorbidities]
_wall = {
    v: widgets.Checkbox(value=False, description=v)
    for v in _vall
}

In [138]:
@interact(
    result=widgets.Dropdown(options=['All', 'Positivo', 'Negativo', 'Inconclusivo']),
    **_wall,
    range_color=widgets.IntRangeSlider(
        value=[0,1000],
        min=0,
        max=1000,
        step=1,
        continuous_update=False,
        orientation='horizontal'
    ),
    radius=widgets.IntSlider(
        value=20,
        min=1,
        max=50,
        step=1,
        continuous_update=False,
        orientation='horizontal'
    ),
)
def show(result: str, range_color: list, radius: int, **_vall):
    aux = df.copy()
    
    if result != 'All':
        aux = aux[aux['RESULTADO'] == result]
    
    _f = [False] * len(aux)
    for p in _vall:
        if _vall[p] is True:
            _f |= aux[p] == 1
        else:
            _f |= aux[p] == 0
    
    aux = aux[_f]
    fig = px.density_mapbox(
        aux,
        lat='lat',
        lon='lon',
        #z='Confirmed',
        radius=radius, # 1
        center=dict(lat=-12.8, lon=-38),
        zoom=8,
        hover_data=[
            'Raça',
            'Sexo',
            'Apresenta sintomas?',
            'Paciente foi hospitalizado?',
            'Paciente foi submetido a ventilação mecânica?',
            'RESULTADO',
        ],
        mapbox_style="carto-positron", # stamen-terrain
    #     animation_frame='DATA DA COLETA',
        range_color=range_color,
        color_continuous_scale=px.colors.sequential.solar_r,
       title=f'Geolocation RT/PCR Test - {len(aux)} records'
    )
    # fig.update_layout(margin={"r":0,"t":30,"l":0,"b":0})
    fig.show()


interactive(children=(Dropdown(description='result', options=('All', 'Positivo', 'Negativo', 'Inconclusivo'), …